# Supplementary Material

This is a companion piece to all the code lines in the paper

Listing: Installation of the package

In [7]:
!pip install metacountregressor --upgrade
!pip install matplotlib --upgrade


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\ahernz\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\ahernz\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Listing: Importing 

In [8]:
from metacountregressor.solution import ObjectiveFunction
from metacountregressor.metaheuristics import (harmony_search,differential_evolution,simulated_annealing)         
from metacountregressor import helperprocess

Listing: Loading in data (example syntax)

In [9]:
import numpy as np
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com"
    "/zahern/data/main/Ex-16-3.csv")
y = df['FREQ']  # Frequency of crashes
X = df.drop(columns=['FREQ']) # setup X based on data
X.columns

Index(['ID', 'LENGTH', 'INCLANES', 'DECLANES', 'WIDTH', 'MIMEDSH', 'MXMEDSH',
       'SPEED', 'URB', 'FC', 'AADT', 'SINGLE', 'DOUBLE', 'TRAIN', 'PEAKHR',
       'GRADEBR', 'MIGRADE', 'MXGRADE', 'MXGRDIFF', 'TANGENT', 'CURVES',
       'MINRAD', 'ACCESS', 'MEDWIDTH', 'FRICTION', 'ADTLANE', 'SLOPE',
       'INTECHAG', 'AVEPRE', 'AVESNOW'],
      dtype='object')

Listing: Defining and Offset Variable

In [10]:
#X['Offset'] = X['LENGTH']*np.log1p(X['AADT']) #Modify Here for Desired Offset
#X = X.drop(columns=['LENGTH', 'AADT'])  
y_name ='FREQ'
offset_name = 'Offset'
group_name = None
panel_name = None

# grabbing the offset amount
X['Offset'] = np.log(1 + X['AADT'] * X['LENGTH'] * 365 / 100000000)
drop_these = ['AADT', 'LENGTH', 'ID','MIMEDSH', 'MXMEDSH', 'SINGLE', 'DOUBLE', 'TRAIN', 'URB']
X.drop(columns=drop_these, inplace=True)

X = X.dropna(how='any')
# ENCODE CATEGORICAL VARIABLES
 # Step 2: Process Data
model_terms = {
    'Y': y_name,         # Dependent variable
    'group': group_name,       # Grouping column (if any)
    'panels': panel_name,      # Panel column (if any)
    'Offset': offset_name       # Offset column (if any)
    }
keep_items = [key for key, value in model_terms.items() if value is not None]
keep_items = keep_items + ['FRICTION', 'SPEED', 'WIDTH']
    # Step 3: Drop highly correlated features and keep required terms
X = helperprocess.interactions(X, keep=keep_items,drop_this_perc=0.65 )
print(X.columns)

# Step 4: Define configuration dictionary
config = {

        'Offset': {'type': 'none'},
        'ADTLANE': {'type': 'none'},
        'FRICTION': {'type': 'normalized'},
        'SPEED':{ 'type': 'normalized'},
        'WIDTH':{ 'type': 'normalized'},
        'MEDWIDTH':{ 'type': 'normalized'},
        'SLOPE':{ 'type': 'normalized'},
        'AVEPRE':{ 'type': 'normalized'},
        'AVESNOW':{ 'type': 'normalized'},
        'INCLANES':{ 'type': 'normalized'},
        'DECLANES':{ 'type': 'normalized'}


}
    
    # Step 5: Infer types for remaining columns and update the configuration
dataset = X

for column in dataset.columns:
    if column not in config:
        print('Guessing type for', column)
        is_binary = dataset[column].isin([0, 1]).all()
        if not is_binary:
            config[column] =  helperprocess.guess_column_type(column,dataset[column])
        else:
            config[column] = {
        'type': None,
        'bins': [0,1],
        'labels': ['Off', 'On'],
        'prefix': f'{column}'}
    else:
        if config[column]['type'] == 'normalized':
            print('Normalizing', column)
            dataset[column] = dataset[column] / dataset[column].max()
            config[column]['type'] = 'none'
    

# Step 6: Transform the dataset based on the configuration
data_new = helperprocess.transform_dataframe(dataset, config)
# Drop duplicate columns by name and keep only the first occurrence
data_new = data_new.loc[:, ~data_new.columns.duplicated()]


print(data_new.head())
data_new = helperprocess.interactions(data_new, keep=keep_items,drop_this_perc=0.65 )
print(data_new.columns)
#reapply correlation check
X = data_new

The following columns are not in the DataFrame and will be ignored: ['Y']
Index(['INCLANES', 'DECLANES', 'WIDTH', 'SPEED', 'FC', 'PEAKHR', 'GRADEBR',
       'MIGRADE', 'MXGRADE', 'MXGRDIFF', 'TANGENT', 'CURVES', 'MINRAD',
       'ACCESS', 'MEDWIDTH', 'FRICTION', 'ADTLANE', 'SLOPE', 'INTECHAG',
       'AVEPRE', 'AVESNOW', 'Offset'],
      dtype='object')
Normalizing INCLANES
Normalizing DECLANES
Normalizing WIDTH
Normalizing SPEED
Guessing type for FC
Guessing type for PEAKHR
made it through here
Guessing type for GRADEBR
made it through here
Guessing type for MIGRADE
Guessing type for MXGRADE
Guessing type for MXGRDIFF
Guessing type for TANGENT
made it through here
Guessing type for CURVES
made it through here
Guessing type for MINRAD
made it through here
Guessing type for ACCESS
Normalizing MEDWIDTH
Normalizing FRICTION
Normalizing SLOPE
Guessing type for INTECHAG
Normalizing AVEPRE
Normalizing AVESNOW
     Offset  ADTLANE  FRICTION     SPEED     WIDTH  MEDWIDTH     SLOPE  \
0  0.0077

customising the arguments for objective and metaheuristic

In [11]:
arguments = {'test_percentage': 0.3, 'complexity_level': 3} #Objective args
arguments_hs = {'_par': 0.3, '_hms': 20}
arguments_sa = None #Note: Supply the relevant arguments, otherwise default arguments will be used
arguments_de = None

Defining the objective function for the available metaheuristics

In [12]:
initial_solution = None
obj_fun = ObjectiveFunction(X, y, **arguments)
#perform harmony search
results_hs = harmony_search(obj_fun, initial_solution, **arguments_hs)
#perform differential evolution
''' Commenting out as only one metaheuristic should be used at a time, feel free to test the others
results_de = differential_evolution(obj_fun, initial_solution, **arguments_de)
#perform simulated annealing
results_sa = simulated_annealing(obj_fun, initial_solution, **arguments_sa)'
'''

no name set, setting name as 0
No Panels. Grouped Random Paramaters Will not be estimated
Setup Complete...
The type of models possible will consider: Poisson, NB
Optimization failed.
Did not converge
--------------------------------------------------------------------------------
Log-Likelihood:  -676.6873566265979
--------------------------------------------------------------------------------
bic: 1522.60
--------------------------------------------------------------------------------
+------------------------+--------+---------+----------+----------+------------+
|         Effect         | $\tau$ |  Coeff  | Std. Err | z-values | Prob |z|>Z |
+========================+========+=========+==========+==========+============+
| const                  | no     | 0.4744  |   3.30   |   0.14   | 0.89       |
+------------------------+--------+---------+----------+----------+------------+
| FRICTION               | no     | 0.1446  |   3.72   |   0.04   | 0.97       |
+--------------------

KeyboardInterrupt: 

Prespecifying an Intitial Solution

In [32]:
manual_fit_spec = {
'fixed_terms': ['const', 'MEDWIDTH', 'SLOPE',  'INCLANES'],
'rdm_terms':  ['SPEED:normal'],
'rdm_cor_terms': [],
'grouped_terms': [],
'hetro_in_means': [],
'transformations': ['no', 'no', 'no', 'no', 'no', 'no', 'no'],
'dispersion': 0
}
arguments['Manual_Fit'] = manual_fit_spec


obj_fun = ObjectiveFunction(X, y, **arguments)
#perform harmony search
#results_hs = harmony_search(obj_fun, , **arguments_hs)

no name set, setting name as 0
No Panels. Grouped Random Paramaters Will not be estimated
Setup Complete...
The type of models possible will consider: Poisson, NB
fitting manual
Benchmaking test with Seed 42
--------------------------------------------------------------------------------
Log-Likelihood:  -727.8796536113379
--------------------------------------------------------------------------------
bic: 1489.80
--------------------------------------------------------------------------------
+------------------------+--------+---------+----------+----------+------------+
|         Effect         | $\tau$ |  Coeff  | Std. Err | z-values | Prob |z|>Z |
+========================+========+=========+==========+==========+============+
| const                  | no     | 1.0006  |   4.24   |   0.24   | 0.81       |
+------------------------+--------+---------+----------+----------+------------+
| MEDWIDTH               | no     | 0.3120  |   2.41   |   0.13   | 0.90       |
+-------------